In [1]:
# 경로설정
import sys
path = "C:\github/aide-analysis/"
path2 = 'C:\github/aide-analysis/notebook/temp_ljs/Develop/지주리포트Lite/실거래 이상값 제거 연구 및 추론/모듈 및 압축파일 보관 폴더'
sys.path.append(path)
sys.path.append(path2)

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd())))
# 한글폰트
import os
import matplotlib.pyplot as plt
if os.name =='posix':
    plt.rc('font',family = 'ApplGothic')
else:
    plt.rc('font',family = 'Malgun Gothic')
    
# 경고출력문 제거
import warnings
warnings.filterwarnings("ignore")

# 모듈 import
import pandas as pd
import src
from src.openapi import KakaoLocalAPI as kakao
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from src.sql import DatabaseHandler
from src.temp import *
from datetime import datetime
import datetime
from dateutil.relativedelta import relativedelta
from src.bigquery import BigqueryHandler
import seaborn as sns
import numpy as np
import statistics
from tqdm import tqdm
import config
from core.config import Config
import json
from datetime import datetime
import geopandas as gpd
from tqdm import tqdm
import folium
from src.bigquery import bigquery_client, bigquery
import geopandas as gpd
from shapely import wkt
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from openpyxl.utils.dataframe import dataframe_to_rows
from difflib import SequenceMatcher
import openpyxl
import math
from 실거래전처리 import TradeProcessing

config = Config()
BH = BigqueryHandler()


# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

# 지난달(23년01월)대비 이번달(23년02월) 매매가격이 떨어진 단지 찾기

- 타입별로 특수성이 있을 수 있기때문에 areaSixGroupNo 대신 pyeongNo를 기준으로 동 타입기준으로 가격비교 하기 -> 이렇게 했을때 문제가 없을지도 검증이 필요하다

In [93]:
# sql = f"""

# SELECT sigunguCode
# FROM `aidepartners.aide.code_sigungu`
# WHERE (isMulti = True AND guLevel = True AND siLevel = False) OR (isMulti = False AND guLevel = True AND siLevel = True)
# """
# sigunguList = BH.read_table(sql)
# sigunguList = list(sigunguList['sigunguCode'])

In [98]:
sql = f"""
SELECT distinct sidoCode
FROM `aidepartners.aide.code_sigungu`
"""
sidoList = BH.read_table(sql)
sidoList = list(sidoList['sidoCode'])

In [108]:
sql =f"""
SELECT T1.*
FROM(
    SELECT sidoCode,
           complexNo, 
           complexName, 
           address,
           areaSixGroupNo, 
           pyeongNo,
           useApproveYear, 
           totalHouseholdCount, 
           tradeType, 
           jeonyongArea,
           supplyArea,
           contractDate, 
           averagePyeong, 
           averagePrice, 
           isFloor, 
           isProcessing1, 
           isProcessing2, 
           isProcessing3, 
           interpolate, 
           yearMonth, 
           isReal, 
           finalProcessing, 
           predict,
           RANK() OVER(PARTITION BY complexNo, complexName, pyeongNo ORDER BY yearMonth DESC) AS rn -- 월 구분
    FROM `aidepartners.raw.aide_apartment_price_origin`
    WHERE -- sidoCode ='28'
          -- sidoCode ='11' 
          yearMonth >='2023-01-01' 
          AND isReal = True
    ORDER BY complexNo, areaSixGRoupNo
    ) AS T1
"""
df = BH.read_table(sql)

In [111]:
data = df.copy()

In [120]:
dropIdx = data[data['sidoCode'].isna()].index
data = data.drop(index = dropIdx)

In [122]:
result = pd.DataFrame()
for sido in sidoList:
    df = data[data['sidoCode']==sido].reset_index(drop = True)
    df_result = pd.DataFrame()
    df_pecent = pd.DataFrame({'per':[]})
    df_unique  = df[['complexNo','pyeongNo','jeonyongArea', 'supplyArea']].drop_duplicates().reset_index(drop =True)
    for i in range(0, len(df_unique)):
        cNo =df_unique['complexNo'].iloc[i]
        typeOption = df_unique['pyeongNo'].iloc[i]
        # 거래가 두달이상 연속으로 있는경우
        if len(df.loc[(df['complexNo']==cNo)&(df['pyeongNo']==typeOption), 'rn'].unique()) >1:
            # 최근거래과 과거거래 금액 차이 비교
            recentPrice = df.loc[(df['complexNo']==cNo)&(df['pyeongNo']==typeOption)&(df['rn']==1),'averagePyeong'].mean()
            pastPrice = df.loc[(df['complexNo']==cNo)&(df['pyeongNo']==typeOption)&(df['rn']==2),'averagePyeong'].mean()
            price = pastPrice - recentPrice
            per = recentPrice / pastPrice # 전달대비 현재 실거래 비율
            if (price >0)&(per<=0.85) : # 전달 거래보다 싸게거래된 경우
                df_result = df_result.append(df[(df['complexNo']==cNo)&(df['pyeongNo']==typeOption)])
                if i ==0:
                    df_result['per'] = ''
                df_result.loc[(df_result['complexNo']==cNo)&(df_result['pyeongNo']==typeOption),'per'] = per
            
                # 결과테이블 세팅
                df_result = df_result[['complexNo','complexName','address','pyeongNo','jeonyongArea', 'supplyArea','per']].drop_duplicates(subset = ['complexNo','pyeongNo']).sort_values(by = 'per')
                df_result['per'] =  round(df_result['per'], 2)

                # 컬럼정리
                df_report = df_result.drop(columns='complexNo').rename(columns = {'complexName':'단지명',
                                                                                  'pyeongNo':'면적타입',
                                                                                  'address':'주소',
                                                                                  'jeonyongArea':'전용면적',
                                                                                  'supplyArea':'공급면적',
                                                                                  'per':'전달 대비 실거래 가격'}).reset_index(drop = True)
                result = result.append(df_report)

In [126]:
result = result.reset_index(drop = True).sort_values(by = '전달 대비 실거래 가격')

In [134]:
result

,단지명,주소,면적타입,전용면적,공급면적,전달 대비 실거래 가격
209,이화(816),경상북도 구미시 인의동,1,44.32,62.09,0.70
62,서동탄역파크자이,경기도 화성시 능동,2,84.97,110.33,0.70
75,서동탄역파크자이,경기도 화성시 능동,2,84.97,110.33,0.70
57,서동탄역파크자이,경기도 화성시 능동,2,84.97,110.33,0.70
53,서동탄역파크자이,경기도 화성시 능동,2,84.97,110.33,0.70
50,서동탄역파크자이,경기도 화성시 능동,2,84.97,110.33,0.70
83,서동탄역파크자이,경기도 화성시 능동,2,84.97,110.33,0.70
194,이화(816),경상북도 구미시 인의동,1,44.32,62.09,0.70
195,이화(816),경상북도 구미시 인의동,1,44.32,62.09,0.70
197,이화(816),경상북도 구미시 인의동,1,44.32,62.09,0.70


In [137]:
result = result.drop_duplicates(subset = ['단지명','면적타입']).reset_index(drop = True)

In [138]:
result

,단지명,주소,면적타입,전용면적,공급면적,전달 대비 실거래 가격
0,이화(816),경상북도 구미시 인의동,1,44.32,62.09,0.70
1,서동탄역파크자이,경기도 화성시 능동,2,84.97,110.33,0.70
2,한신,경상북도 김천시 황금동,2,84.97,103.04,0.70
3,삼도한솔타운3차,경상북도 포항시 남구 연일읍,3,84.81,98.94,0.72
4,세중세미클래식,충청남도 논산시 취암동,1,36.29,53.30,0.72
5,청주개신주공1단지,충청북도 청주시 서원구 개신동,2,59.90,85.57,0.72
6,주공2단지,대전시 중구 문화동,1,49.77,71.65,0.74
7,세종모닝시티2차(도시형),세종시 세종시 나성동,1,23.28,36.34,0.74
8,금부마을호반리젠시빌3차,광주시 서구 금호동,1,84.75,109.19,0.76
9,북실,강원도 정선군 정선읍,1,39.30,45.15,0.76
